In [35]:
import tensorflow

In [36]:
import keras

In [44]:
print(1)

1


In [38]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [39]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2333726643571254668
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11281553818
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10185625500477188138
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [40]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/data/tb-logs', histogram_freq=0, write_graph=True, write_images=True)


In [45]:
img_width, img_height = 256, 256
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [46]:
nr_categories = 102
batch_size = 32
epochs = 50

In [65]:
k.set_learning_phase(0)
base_model = applications.inception_resnet_v2.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#from keras.models import load_model
#model = load_model("/data/vgg16_fungi_all_p2.h5")

In [66]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_6
1 conv2d_407
2 batch_normalization_408
3 activation_407
4 conv2d_408
5 batch_normalization_409
6 activation_408
7 conv2d_409
8 batch_normalization_410
9 activation_409
10 max_pooling2d_9
11 conv2d_410
12 batch_normalization_411
13 activation_410
14 conv2d_411
15 batch_normalization_412
16 activation_411
17 max_pooling2d_10
18 conv2d_415
19 batch_normalization_416
20 activation_415
21 conv2d_413
22 conv2d_416
23 batch_normalization_414
24 batch_normalization_417
25 activation_413
26 activation_416
27 average_pooling2d_3
28 conv2d_412
29 conv2d_414
30 conv2d_417
31 conv2d_418
32 batch_normalization_413
33 batch_normalization_415
34 batch_normalization_418
35 batch_normalization_419
36 activation_412
37 activation_414
38 activation_417
39 activation_418
40 mixed_5b
41 conv2d_422
42 batch_normalization_423
43 activation_422
44 conv2d_420
45 conv2d_423
46 batch_normalization_421
47 batch_normalization_424
48 activation_420
49 activation_423
50 conv2d_419
51 conv2d_421
52 conv2d_42

In [67]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_407 (Conv2D)             (None, 127, 127, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_408 (BatchN (None, 127, 127, 32) 96          conv2d_407[0][0]                 
__________________________________________________________________________________________________
activation_407 (Activation)     (None, 127, 127, 32) 0           batch_normalization_408[0][0]    
__________________________________________________________________________________________________
conv2d_408

block17_3_conv (Conv2D)         (None, 14, 14, 1088) 418880      block17_3_mixed[0][0]            
__________________________________________________________________________________________________
block17_3 (Lambda)              (None, 14, 14, 1088) 0           block17_2_ac[0][0]               
                                                                 block17_3_conv[0][0]             
__________________________________________________________________________________________________
block17_3_ac (Activation)       (None, 14, 14, 1088) 0           block17_3[0][0]                  
__________________________________________________________________________________________________
conv2d_496 (Conv2D)             (None, 14, 14, 128)  139264      block17_3_ac[0][0]               
__________________________________________________________________________________________________
batch_normalization_497 (BatchN (None, 14, 14, 128)  384         conv2d_496[0][0]                 
__________

In [68]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for layer in base_model.layers[:]:
    print(layer.name)
    layer.trainable = False

input_6
conv2d_407
batch_normalization_408
activation_407
conv2d_408
batch_normalization_409
activation_408
conv2d_409
batch_normalization_410
activation_409
max_pooling2d_9
conv2d_410
batch_normalization_411
activation_410
conv2d_411
batch_normalization_412
activation_411
max_pooling2d_10
conv2d_415
batch_normalization_416
activation_415
conv2d_413
conv2d_416
batch_normalization_414
batch_normalization_417
activation_413
activation_416
average_pooling2d_3
conv2d_412
conv2d_414
conv2d_417
conv2d_418
batch_normalization_413
batch_normalization_415
batch_normalization_418
batch_normalization_419
activation_412
activation_414
activation_417
activation_418
mixed_5b
conv2d_422
batch_normalization_423
activation_422
conv2d_420
conv2d_423
batch_normalization_421
batch_normalization_424
activation_420
activation_423
conv2d_419
conv2d_421
conv2d_424
batch_normalization_420
batch_normalization_422
batch_normalization_425
activation_419
activation_421
activation_424
block35_1_mixed
block35_1_conv

activation_587
conv2d_588
batch_normalization_589
activation_588
conv2d_586
conv2d_589
batch_normalization_587
batch_normalization_590
activation_586
activation_589
block8_5_mixed
block8_5_conv
block8_5
block8_5_ac
conv2d_591
batch_normalization_592
activation_591
conv2d_592
batch_normalization_593
activation_592
conv2d_590
conv2d_593
batch_normalization_591
batch_normalization_594
activation_590
activation_593
block8_6_mixed
block8_6_conv
block8_6
block8_6_ac
conv2d_595
batch_normalization_596
activation_595
conv2d_596
batch_normalization_597
activation_596
conv2d_594
conv2d_597
batch_normalization_595
batch_normalization_598
activation_594
activation_597
block8_7_mixed
block8_7_conv
block8_7
block8_7_ac
conv2d_599
batch_normalization_600
activation_599
conv2d_600
batch_normalization_601
activation_600
conv2d_598
conv2d_601
batch_normalization_599
batch_normalization_602
activation_598
activation_601
block8_8_mixed
block8_8_conv
block8_8
block8_8_ac
conv2d_603
batch_normalization_604


In [75]:
#Adding custom Layers 
k.set_learning_phase(1)
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x, training=False)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

In [76]:
print(1)

1


In [58]:
from keras.layers.normalization import BatchNormalization

In [64]:
#Adding custom Layers 
k.set_learning_phase(1)
#input = keras.layers.Input(shape=(256, 256, 3))
#x = base_model(input)
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
#x = Dense(102, activation="relu")(x)
#x = BatchNormalization()(x, training=False)
predictions = Dense(nr_categories, activation="softmax")(x)

AttributeError: Layer inception_resnet_v2 has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [77]:
model_final = Model(input = base_model.input, output = predictions)
#model_final = model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [ ]:
model_final.shape

In [96]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for i, layer in enumerate(model_final.layers):
    print(i, layer.name)
#    layer.trainable = False

0 conv2d_606
1 conv2d_609
2 batch_normalization_607
3 batch_normalization_610
4 activation_606
5 activation_609
6 block8_10_mixed
7 block8_10_conv
8 block8_10
9 conv_7b
10 conv_7b_bn
11 conv_7b_ac


In [14]:
# model_final.summary()

In [78]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [102]:
train__val_datagen = ImageDataGenerator(#rescale = 1./255, 
                                        rotation_range=40,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest',
                                        preprocessing_function=keras.applications.resnet50.preprocess_input, 
                                        validation_split=0.2)

In [103]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [104]:
img_width, img_height = 256, 256
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 4604 images belonging to 102 classes.


In [105]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 1094 images belonging to 102 classes.


In [108]:
# Save the model according to the conditions  
import time
t = time.time()
checkpoint = ModelCheckpoint("inception_resnet_v2_oxford102_train_dataset_{}.h5".format(str(t)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [109]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [84]:
#batch 16

In [ ]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
144/143 [==============================] - 149s 1s/step - loss: 4.4040 - acc: 0.0962 - val_loss: 3.8118 - val_acc: 0.1554

Epoch 00001: val_acc improved from -inf to 0.15539, saving model to inception_resnet_v2_oxford102_train_dataset.h5
Epoch 2/50
144/143 [==============================] - 137s 954ms/step - loss: 3.5080 - acc: 0.2181 - val_loss: 3.3195 - val_acc: 0.2495

Epoch 00002: val_acc improved from 0.15539 to 0.24954, saving model to inception_resnet_v2_oxford102_train_dataset.h5
Epoch 3/50
144/143 [==============================] - 140s 972ms/step - loss: 3.0976 - acc: 0.2818 - val_loss: 3.0704 - val_acc: 0.2733

Epoch 00003: val_acc improved from 0.24954 to 0.27331, saving model to inception_resnet_v2_oxford102_train_dataset.h5
Epoch 4/50
144/143 [==============================] - 139s 966ms/step - loss: 2.7778 - acc: 0.3428 - val_loss: 2.7417 - val_acc: 0.3473

Epoch 00004: val_acc improved from 0.27331 to 0.34735, saving model to inception_resnet_v2_oxford102_tra

In [ ]:
for i, layer in enumerate(base_model.layers[:768]):
    #print(i, layer.name)
    layer.trainable = False
for i, layer in enumerate(base_model.layers[768:]):
    #print(i, layer.name)
    layer.trainable = False


In [99]:
for i, layer in enumerate(base_model.layers[:768]):
    #print(i, layer.name)
    layer.trainable = False
for i, layer in enumerate(base_model.layers[768:]):
    #print(i, layer.name)
    layer.trainable = True

In [100]:
model = Model(input = base_model.input, output = predictions)
#model_final = model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [101]:
# compile the model 
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [111]:
model.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
144/143 [==============================] - 150s 1s/step - loss: 5.2273 - acc: 0.0230 - val_loss: 4.6671 - val_acc: 0.0219

Epoch 00001: val_acc improved from 0.00548 to 0.02194, saving model to inception_resnet_v2_oxford102_train_dataset_1542060442.2020257.h5
Epoch 2/50
144/143 [==============================] - 138s 959ms/step - loss: 4.6072 - acc: 0.0319 - val_loss: 4.6112 - val_acc: 0.0293

Epoch 00002: val_acc improved from 0.02194 to 0.02925, saving model to inception_resnet_v2_oxford102_train_dataset_1542060442.2020257.h5
Epoch 3/50
144/143 [==============================] - 138s 961ms/step - loss: 4.5495 - acc: 0.0374 - val_loss: 4.5316 - val_acc: 0.0448

Epoch 00003: val_acc improved from 0.02925 to 0.04479, saving model to inception_resnet_v2_oxford102_train_dataset_1542060442.2020257.h5
Epoch 4/50
144/143 [==============================] - 139s 968ms/step - loss: 4.5292 - acc: 0.0328 - val_loss: 4.5248 - val_acc: 0.0302

Epoch 00004: val_acc did not improve from 0.